<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/model_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model explainability example

* The new code related to model explainability starts half-way through, after the classifier has been trained


# Train BERT text classifier using the Transformers library

This notebook shows how to train a simple text classifier by fine-tuning a pre-trained BERT model using the Hugging Face [Transformers](https://huggingface.co/transformers/) library.

This notebook is based in part on the [Text classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb) notebook.

**NOTE**: it's recommended to run this using a runtime with a GPU. Select "Runtime" -> "Change runtime type" from the top menu in Colab and set "Hardware accelerator" to "GPU" when starting

## Install libraries

First, we'll use [`pip`](https://pypi.org/project/pip/) to install two Python libraries that are used in this notebook: [`transformers`](https://huggingface.co/transformers/) and [`datasets`](https://huggingface.co/docs/datasets/).

In [1]:
!pip --quiet install transformers
!pip --quiet install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

## Set model, dataset and hyperparameters

Let's then set some global variables such as the name of the pre-trained model and the hyperparameters we'll use for fine-tuning it.

* `MODEL_NAME`: the name of a pretrained model included in the [model repository](https://huggingface.co/models)
* `DATASET`: the path and name of a dataset included in the [dataset repository](https://huggingface.co/datasets)
* `LEARNING_RATE`, `BATCH_SIZE`, and `TRAIN_EPOCHS`: hyperparameters to use for fine-turning the model. (Try different values here!)

Here, we'll use the [Stanford Sentiment Treebank](https://huggingface.co/datasets/sst) dataset as prepared for the [GLUE](https://huggingface.co/datasets/glue) collection. This is a binary text classification task where the objective is to determine if sentences express a positive or negative sentiment.

In [3]:
MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

## Load dataset

We'll first load the dataset with [`load_dataset`](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset) and determine the number of distinct labels in the (training) data.

In [4]:
dataset = load_dataset(*DATASET)
num_labels = len(set(dataset['train']['label']))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

The loaded dataset is a simple dictionary-like container for distinct `DataSet` objects for training, development (validation), and test data:

In [5]:
print(dataset)
print(f'number of distinct labels: {num_labels}')

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
number of distinct labels: 2


To reduce training time a bit, let's just take every 10th item in the `train` subset of the dataset.

In [6]:
# I leave this undone to get the best possible model later for the explainability part
#dataset['train'] = dataset['train'].filter(lambda example, idx: idx % 10 == 0, with_indices=True)

## Load tokenizer and tokenize data

We'll then load an appropriate tokenizer for the pre-trained model we'll be using with [`AutoTokenizer.from_pretrained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoTokenizer.from_pretrained)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

The tokenizer can take a string (or e.g. a list of strings) and performs all the necessary preprocessing steps to prepare its input for use by the model, including splitting input texts into tokens, adding padding and special characters, and mapping those to integer IDs (`input_ids`). The tokenizer also creates attention weights (`attention_mask`) for the model.

We'll then define an encoding function applying the tokenizer to the text data  (key `"sentence"`) of a [`Dataset`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=dataset#datasets.Dataset) object, and use the `map` function of the `DatasetDict` to tokenize the train, development, and test datasets.

In [8]:
def encode_dataset(d):
  return tokenizer(d['sentence'])

encoded_dataset = dataset.map(encode_dataset)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

## Load pre-trained model

Next, we'll load the pre-trained model with support for text classification output using [`AutoModelForSequenceClassification.from_prertained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoModelForSequenceClassification.from_pretrained).

Note that we need to provide the number of labels in the data when loading the model.

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training parameters and metrics
We're almost ready to train. We'll next create a [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) object to hold the hyperparameters and other settings that are appropriate for training on Colab:

* `save_strategy`: set to `"epoch"` so that model checkpoints are saved every epoch, this I can do because I save to own Google Drive
* `evaluation_strategy` and `logging_strategy` set to `"epoch"` so that evaluation and logging are performed once per epoch
* The hyperparameters `LEARNING_RATE`, `BATCH_SIZE` and `TRAIN_EPOCHS` set above are passed to the training process through this object

Finally, we'll define a simple accuracy metric measuring how many predictions match their correct values.

In [10]:
def compute_accuracy(pred):
    y_pred = pred.predictions.argmax(axis=1)
    y_true = pred.label_ids
    return { 'accuracy': sum(y_pred == y_true) / len(y_true) }

# Training (fine-tuning)

For fine-tuning the pre-trained model, we'll create a [`Trainer`](https://huggingface.co/transformers/main_classes/trainer.html) object, providing it with the pre-trained model, settings, training and development (validation) data, and the evaluation metric created above.

In [11]:


# Hyperparameters
LEARNING_RATE=1e-4
BATCH_SIZE=128
TRAIN_EPOCHS=4

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import wandb
wandb.init(mode="disabled")

train_args = TrainingArguments(
    '/content/drive/MyDrive/WorkStuff/ssd2-sentiment-model',    # output directory for checkpoints and predictions
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    logging_strategy='steps',
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=TRAIN_EPOCHS
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = Trainer(
      model,
      train_args,
      train_dataset=encoded_dataset['train'],
      eval_dataset=encoded_dataset['validation'],
      tokenizer=tokenizer,
      compute_metrics=compute_accuracy
)

<ipython-input-24-3ad0df25d89a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training is then performed simply by calling the `train` function of the `Trainer` object.

In [25]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.090600,0.318958,0.894495
1000,0.090400,0.232974,0.913991
1500,0.051400,0.308563,0.905963
2000,0.027700,0.379153,0.902523


TrainOutput(global_step=2108, training_loss=0.06289136839546335, metrics={'train_runtime': 2186.9604, 'train_samples_per_second': 123.183, 'train_steps_per_second': 0.964, 'total_flos': 7153427914801860.0, 'train_loss': 0.06289136839546335, 'epoch': 4.0})

## Evaluation and predictions for user input

We can use `trainer.evaluate()` to evaluate the trained model on the `eval_dataset` given to the trainer:

In [26]:
results = trainer.evaluate()
print(f'Accuracy: {results["eval_accuracy"]}')

Accuracy: 0.9139908256880734


# Model save

* Now that the model is trained, we may want to save it to Google Drive
* That way we don't need to train it every time

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Oh no! Old code, don't save like this! Use the trainer!

import torch
#make sure you make this directory
torch.save(trainer.model,"/content/drive/MyDrive/WorkStuff/sent_model.pt")

# Model load

* Now we load the model and repeat some of the imports so it is possible to run the notebook from this point onwards

In [29]:
# relevant stuff repeated from above so you can run it from here onwards if you happen to have the model trained

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

try that out

# Model explainability with the captum library

* captum.ai is a nifty library with many of the explainability algorithms implemented
* we will use it here to try the techniques
* this is not the easiest of code, btw, many of the libraries are bleeding-edge, so you cut yourself here and there (the blood on the edge is yours)
* the code below is (quite substantial) adaptation of https://captum.ai/tutorials/Bert_SQUAD_Interpret

In [30]:
!pip install captum pandas matplotlib seaborn transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import torch
import transformers
tokenizer=transformers.AutoTokenizer.from_pretrained("bert-base-cased")
model = torch.load("/content/drive/MyDrive/WorkStuff/sent_model.pt")
model.to('cpu')    # simplifies input placement

sentiment_label = [
    'negative',
    'positive'
]

def predict_sentiment(string):
    tokenized = tokenizer(string, return_tensors='pt')
    pred = model(**tokenized)
    pred_idx = pred.logits.detach().numpy().argmax()
    return sentiment_label[pred_idx]

<ipython-input-32-836635ec4b4e>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/drive/MyDrive/WorkStuff/sent_model.pt")


In [33]:
example_sentences = [
    'I think this is very good, and I am in support.',
    'This is bad news.',
    'This could not have been better.',
    'This is not good news at all.',
    'Donald Trump lost the election and will no longer be every day on CNN.',
    'I think your service fulfilled my expectations, I could not really expect you to be any better'
]

for e in example_sentences:
    print(e, '->', predict_sentiment(e))

I think this is very good, and I am in support. -> positive
This is bad news. -> negative
This could not have been better. -> positive
This is not good news at all. -> negative
Donald Trump lost the election and will no longer be every day on CNN. -> negative
I think your service fulfilled my expectations, I could not really expect you to be any better -> negative


In [34]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [35]:
#Tells the model that it is in evaluation mode, and zeroes out the gradients
model.eval()
model.zero_grad()

# Helper functions

In [36]:
# Forward on the model -> data in, prediction out, nothing fancy really
def predict(inputs, token_type_ids, attention_mask):
    pred=model(inputs, token_type_ids=token_type_ids, attention_mask=attention_mask)
    return pred.logits #return the output of the classification layer




In [37]:
# Given input text, construct a pair of (text input, blank reference input as long as the text itself)
def construct_input_ref_pair(text):
    inp=tokenizer(text,return_tensors="pt")
    #let's have our blank be all padding
    #but how long is the text?
    #one way to do this:
    tokenized=tokenizer(text,add_special_tokens=False)["input_ids"]
    blank_input=" ".join(["[PAD]"]*len(tokenized)) #and now make an input with this many [PAD] tokens
    ref=tokenizer(blank_input,return_tensors="pt")
    return (inp["input_ids"],inp["token_type_ids"],inp["attention_mask"]),\
    (ref["input_ids"],ref["token_type_ids"],ref["attention_mask"])

In [38]:
# Let's try it!

text="This piece of news is great, even if you think it is rather unfortunate"

#input:
inp, ref = construct_input_ref_pair(text)
all_tokens = tokenizer.convert_ids_to_tokens(inp[0][0])
print(all_tokens)

['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'great', ',', 'even', 'if', 'you', 'think', 'it', 'is', 'rather', 'unfortunate', '[SEP]']


In [39]:
p=predict(*inp)
print("p=",p)
print("p.shape",p.shape)

p= tensor([[-2.9966,  3.0065]], grad_fn=<AddmmBackward0>)
p.shape torch.Size([1, 2])


In [40]:
# Yay, now we finally made it to the attribution part
lig = LayerIntegratedGradients(predict, model.bert.embeddings) #attribute the output wrt to embeddings

In [41]:
# inputs: inputs
# baselines: the blank baseline
# target: which of the two classes in the output (pos/neg) to run the prediction against?
attrs, delta = lig.attribute(inputs=inp,
                                  baselines=ref,
                                  return_convergence_delta=True,target=1)
print("attrs shape",attrs.shape)

attrs shape torch.Size([1, 17, 768])


In [42]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.vector_norm(attributions,dim=0)
    return attributions

attrs_sum = summarize_attributions(attrs)
print("attrs_sum shape",attrs_sum.shape)

attrs_sum shape torch.Size([17])


In [43]:
print(attrs_sum)
print(tokenizer.convert_ids_to_tokens(inp[0][0]))

for a,t in zip(attrs_sum,tokenizer.convert_ids_to_tokens(inp[0][0])):
    print(float(a),t)

tensor([ 0.0000, -0.0429,  0.0323, -0.1013,  0.1141,  0.1879,  0.9275, -0.1052,
         0.1696,  0.1419,  0.0395, -0.0644, -0.0706, -0.0571, -0.0625, -0.0199,
         0.0000], dtype=torch.float64)
['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'great', ',', 'even', 'if', 'you', 'think', 'it', 'is', 'rather', 'unfortunate', '[SEP]']
0.0 [CLS]
-0.04292610011129043 This
0.03225936394030801 piece
-0.10132114418438082 of
0.1140787070862297 news
0.1878815129664638 is
0.9275410590326545 great
-0.1051501821090733 ,
0.16961629844110984 even
0.1418772415461568 if
0.03954609326534221 you
-0.06439117580902615 think
-0.07061663970163856 it
-0.057055036265383684 is
-0.06247256788764523 rather
-0.01986332574986341 unfortunate
0.0 [SEP]


Damn, that seems to work!

In [44]:
import captum
from IPython.core.display import HTML, display
x=captum.attr.visualization.format_word_importances(tokenizer.convert_ids_to_tokens(inp[0][0]),attrs_sum)
HTML(x)

# Almost there...

* Let's wrap this all into a function

In [45]:


def predict_and_explain(model,text):
    model.zero_grad() #to be safe perhaps it's not needed
    inp,ref = construct_input_ref_pair(text)
    all_tokens = tokenizer.convert_ids_to_tokens(inp[0][0])

    lig = LayerIntegratedGradients(predict, model.bert.embeddings)
    prediction=predict(*inp)[0]
    prediction_cls=int(torch.argmax(prediction))
    print("Prediction:", ("negative","positive")[prediction_cls],"Weights:",prediction.tolist())
    for target,classname in enumerate(("negative","positive")):

        attrs, delta = lig.attribute(inputs=inp,
                                  baselines=ref,
                                  return_convergence_delta=True,target=target)
        attrs_sum = summarize_attributions(attrs)

        x=captum.attr.visualization.format_word_importances(all_tokens,attrs_sum)
        print("ATTRIBUTION WITH RESPECT TO",classname)
        display(HTML(x))
        print()

In [46]:
predict_and_explain(model,"I like ChatGPT. It is an awesome technology, even though I see some dangerous aspects too.")

Prediction: positive Weights: [-3.1852171421051025, 3.402066230773926]
ATTRIBUTION WITH RESPECT TO negative



ATTRIBUTION WITH RESPECT TO positive
